In [1]:
import pandas as pd
import numpy as np

import os
import glob
os.chdir('./Colegios')

Primero, leemos cada archivo de cada departamento y eliminamos todas aquellas fillas que sean en blanco.

In [2]:
#all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
all_filenames = []
for i in glob.glob('*.{}'.format('csv')):
    f = open(i)
    newDoc = open('temp/temp_'+i, 'w+')
    for index, line in enumerate(f):
        if index != 0:
            newDoc.write(line)
    file = pd.read_csv('temp/temp_'+i)
    all_filenames.append(file)
len(all_filenames)

23

Combinamos todos los archivos csv en uno solo debido a que todos los archivos cuentan con las mismas variables.

In [3]:
combined_csv = None
combined_csv = pd.concat([f for f in all_filenames ])
combined_csv.to_csv( "combined_csv.csv", index=False)

Realizamos una exploración rapida del dataset

In [4]:
combined_csv.head()

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,09-01-0040-46,09-041,QUETZALTENANGO,QUETZALTENANGO,"COLELGIO EVANGELICO ""LA PATRIA""",19 AVENIDA 1-28 ZONA 3,NaN,WILSON RANFERI SIGUENZA LOPEZ,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,DIARIO(REGULAR),QUETZALTENANGO
1,09-01-0098-46,09-006,QUETZALTENANGO,QUETZALTENANGO,"COLEGIO PRIVADO MIXTO ""SUIZO QUETZALTENANGO""",1RA CALLE 43-50 ZONA 8,79267367,PEDRO GERсNIMO SARAT CHAJ,MIRNA LETICIA CIFUENTES GODINEZ DE CIFUENTES,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),QUETZALTENANGO
2,09-01-0125-46,09-005,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO NACIONAL DE BACHILLERATO POR MADUREZ...,"4A. CALLE 25-20, ZONA 3",41514439,JUAN CARLOS RONQUILLO DE LEON,NaN,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),QUETZALTENANGO
3,09-01-0126-46,09-005,QUETZALTENANGO,QUETZALTENANGO,ESCUELA NACIONAL DE CIENCIAS COMERCIALES 'PROF...,5A CALLE 12-60 ZONA 3,50171669,JUAN CARLOS RONQUILLO DE LEON,NaN,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),QUETZALTENANGO
4,09-01-0131-46,09-005,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO NORMAL PARA VARONES DE OCCIDENTE I.N...,7MA. CALLE 29-25 ZONA 3,77611583,JUAN CARLOS RONQUILLO DE LEON,FRANCISCO MIGUEL TIRADO HUINAC,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO


In [5]:
combined_csv.count()

CODIGO             7617
DISTRITO           7408
DEPARTAMENTO       7617
MUNICIPIO          7617
ESTABLECIMIENTO    7617
DIRECCION          7564
TELEFONO           7066
SUPERVISOR         7407
DIRECTOR           6006
NIVEL              7617
SECTOR             7617
AREA               7617
STATUS             7617
MODALIDAD          7617
JORNADA            7617
PLAN               7617
DEPARTAMENTAL      7617
dtype: int64

In [6]:
combined_csv.dtypes

CODIGO             object
DISTRITO           object
DEPARTAMENTO       object
MUNICIPIO          object
ESTABLECIMIENTO    object
DIRECCION          object
TELEFONO           object
SUPERVISOR         object
DIRECTOR           object
NIVEL              object
SECTOR             object
AREA               object
STATUS             object
MODALIDAD          object
JORNADA            object
PLAN               object
DEPARTAMENTAL      object
dtype: object

# Duplicados

Verificamos si en el dataset cuenta con filas que son exactamente iguales (duplicadas), es decir, que todos los campos de la fila sean identicas a otras.

In [7]:
combined_csv.duplicated().sum()

0

Debido a que no hay filas exactamente iguales, verificamos si hay establecimientos que se llaman de la misma forma y efectivamente nos topamos con registros duplicados.

In [8]:
combined_csv.ESTABLECIMIENTO.duplicated().sum()

3024

In [9]:
duplicates = combined_csv[combined_csv['ESTABLECIMIENTO'].duplicated(keep=False)]['ESTABLECIMIENTO'].tolist()
duplicates[:10]

['INSTITUTO NORMAL PARA SEяORITAS DE OCCIDENTE',
 'INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFICADA COLEGIO CORAZON DE JESUS',
 "COLEGIO MIXTO BILINGUE 'SAN PABLO'",
 'COLEGIO EVANGELICO LA PATRIA',
 'INSTITUTO TECNOLсGICO PRIVADO DE OCCIDENTE',
 'INSTITUTO PRIVADO MIXTO DE OCCIDENTE',
 'INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFICADA COLEGIO CORAZON DE JESUS',
 'COLEGIO PRIVADO MIXTO "LICEO EDUCACIONAL LIVERMORE"',
 'COLEGIO MIXTO PRIVADO SAN JOSи',
 'COLEGIO PRIVADO MIXTO "LICEO EDUCACIONAL LIVERMORE"']

Eliminamos todos aquellos establecimientos que se encuentran en el mismo municipio y mismo distrito y mantenemos el ultimo registro para no desaparecer el establecimiento.

In [10]:
# set_duplicates = set(duplicates)
# for r in set_duplicates:
#     dup = combined_csv[combined_csv['ESTABLECIMIENTO'] == r] and combined_csv[combined_csv['MUNICIPIO'] == r]
#     for i in dup.index:
#       if i in combined_csv.index:
#         combined_csv = combined_csv.drop(i)

combined_csv = combined_csv.drop_duplicates(subset=['ESTABLECIMIENTO','MUNICIPIO','DISTRITO'], keep='last')

# Valores nulos

Verificamos la cantidad de datos nulos que tenemos en el dataset

In [11]:
combined_csv.isnull().sum()

CODIGO                0
DISTRITO            199
DEPARTAMENTO          0
MUNICIPIO             0
ESTABLECIMIENTO       0
DIRECCION            53
TELEFONO            503
SUPERVISOR          200
DIRECTOR           1410
NIVEL                 0
SECTOR                0
AREA                  0
STATUS                0
MODALIDAD             0
JORNADA               0
PLAN                  0
DEPARTAMENTAL         0
dtype: int64

Para todos los datos nulos que, como podemos observar, en su mayoria se encuentran en direccion, distrito, telefono, director y supervisor, en lugar de eliminar todas las filas, ya que estos datos nulos representan mas del 1.5%, creamos una categoria de 'SIN ESPECIFICAR' y en el distrito '00-000'.

In [12]:
combined_csv.DISTRITO = combined_csv.DISTRITO.fillna('00-000')

In [13]:
combined_csv.DIRECCION = combined_csv.DIRECCION.fillna('SIN ESPECIFICAR')

In [14]:
combined_csv.TELEFONO = combined_csv.TELEFONO.fillna('SIN ESPECIFICAR')

In [15]:
combined_csv.DIRECTOR = combined_csv.DIRECTOR.fillna('SIN ESPECIFICAR')

In [16]:
combined_csv.SUPERVISOR = combined_csv.SUPERVISOR.fillna('SIN ESPECIFICAR')

Seleccionamos todas las celdas que unicamente contienen . y - ya que eston representan mas celdas vacias y tambien las completamos con su respectivo 'SIN ESPECIFICAR'

In [17]:
hidden_NaN = combined_csv.DIRECTOR.str.contains('[.-]')

In [18]:
combined_csv.loc[hidden_NaN].head()

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,09-01-0040-46,09-041,QUETZALTENANGO,QUETZALTENANGO,"COLELGIO EVANGELICO ""LA PATRIA""",19 AVENIDA 1-28 ZONA 3,SIN ESPECIFICAR,WILSON RANFERI SIGUENZA LOPEZ,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,DIARIO(REGULAR),QUETZALTENANGO
18,09-01-0149-46,09-,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,9A AVE. D4-71 ZONA 1,SIN ESPECIFICAR,WILSON RANFERI SIGUENZA LOPEZ,.,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),QUETZALTENANGO
32,09-01-0198-46,09-005,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,7MA. CALLE 29-25 ZONA 3,42542749,JUAN CARLOS RONQUILLO DE LEON,-,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO
34,09-01-0221-46,09-005,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ...,AVENIDA LAS AMERICAS 9-80 ZONA 9,77632393,JUAN CARLOS RONQUILLO DE LEON,---,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO
144,09-01-3336-46,09-002,QUETZALTENANGO,QUETZALTENANGO,COLEGIO PRIVADO MIXTO 'LICEO SANTA MARIA',16 AVENIDA 8-52 A ZONA 1,78242068,ROSEMERY GUADALUPE PIEDRASANTA GRAMAJO DE SIGU...,.,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,FIN DE SEMANA,QUETZALTENANGO


In [19]:
h = combined_csv.loc[hidden_NaN]

In [20]:
len(combined_csv.DIRECTOR[hidden_NaN])

335

In [21]:
for i in list(h.index):
    combined_csv['DIRECTOR'][i] = 'SIN ESPECIFICAR'

In [22]:
hidden_NaN = combined_csv.DIRECTOR.str.contains('[.-]')

In [23]:
len(combined_csv.DIRECTOR[hidden_NaN])

0

Verificamos que ya no existan celdas vacias en el dataset

In [24]:
combined_csv.isnull().sum()

CODIGO             0
DISTRITO           0
DEPARTAMENTO       0
MUNICIPIO          0
ESTABLECIMIENTO    0
DIRECCION          0
TELEFONO           0
SUPERVISOR         0
DIRECTOR           0
NIVEL              0
SECTOR             0
AREA               0
STATUS             0
MODALIDAD          0
JORNADA            0
PLAN               0
DEPARTAMENTAL      0
dtype: int64

# Filtros

Existen registros en la variable de telefono que tienen dos numeros registrados, se identifican esos campos y dejamos el primer numero telefonico.

In [25]:
combined_csv["TELEFONO"] = combined_csv["TELEFONO"].astype(str).str.split('-', expand = True)[0]

In [26]:
combined_csv['TELEFONO'] = combined_csv['TELEFONO'].str[0:8]

# Caractéres Especiales

Reemplazamos todos los caracteres especiales a sus respectivo caracter en el español.

In [27]:
combined_csv = combined_csv.replace('а','Á', regex=True)
combined_csv = combined_csv.replace('и','É', regex=True)
combined_csv = combined_csv.replace('м','Í', regex=True)
combined_csv = combined_csv.replace('с','Ó', regex=True)
combined_csv = combined_csv.replace('з','Ú', regex=True)
combined_csv = combined_csv.replace('э','Ü', regex=True)
combined_csv = combined_csv.replace('я','Ñ', regex=True)

In [28]:
combined_csv.head(20)

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,09-01-0040-46,09-041,QUETZALTENANGO,QUETZALTENANGO,"COLELGIO EVANGELICO ""LA PATRIA""",19 AVENIDA 1-28 ZONA 3,SIN ESPE,WILSON RANFERI SIGUENZA LOPEZ,SIN ESPECIFICAR,DIVERSIFICADO,PRIVADO,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,DOBLE,DIARIO(REGULAR),QUETZALTENANGO
1,09-01-0098-46,09-006,QUETZALTENANGO,QUETZALTENANGO,"COLEGIO PRIVADO MIXTO ""SUIZO QUETZALTENANGO""",1RA CALLE 43-50 ZONA 8,79267367,PEDRO GERÓNIMO SARAT CHAJ,MIRNA LETICIA CIFUENTES GODINEZ DE CIFUENTES,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),QUETZALTENANGO
2,09-01-0125-46,09-005,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO NACIONAL DE BACHILLERATO POR MADUREZ...,"4A. CALLE 25-20, ZONA 3",41514439,JUAN CARLOS RONQUILLO DE LEON,SIN ESPECIFICAR,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),QUETZALTENANGO
3,09-01-0126-46,09-005,QUETZALTENANGO,QUETZALTENANGO,ESCUELA NACIONAL DE CIENCIAS COMERCIALES 'PROF...,5A CALLE 12-60 ZONA 3,50171669,JUAN CARLOS RONQUILLO DE LEON,SIN ESPECIFICAR,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),QUETZALTENANGO
4,09-01-0131-46,09-005,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO NORMAL PARA VARONES DE OCCIDENTE I.N...,7MA. CALLE 29-25 ZONA 3,77611583,JUAN CARLOS RONQUILLO DE LEON,SIN ESPECIFICAR,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO
6,09-01-0134-46,09-002,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,AVE. JESUS CASTILLO A-74 ZONA 1,77610735,ROSEMERY GUADALUPE PIEDRASANTA GRAMAJO DE SIGU...,SIN ESPECIFICAR,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO
7,09-01-0135-46,09-036,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,7A AVE. 6-45 ZONA 1,77614475,OBDULIO ROLANDO RUIZ ALVA,SIN ESPECIFICAR,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),QUETZALTENANGO
8,09-01-0136-46,09-036,QUETZALTENANGO,QUETZALTENANGO,COLEGIO MIXTO TERESA MARTIN,9A CALLE 10-09 ZONA 1,77612237,OBDULIO ROLANDO RUIZ ALVA,SIN ESPECIFICAR,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),QUETZALTENANGO
9,09-01-0137-46,09-004,QUETZALTENANGO,QUETZALTENANGO,COLEGIO EVANGELICO 'BETHANIA',CALLE C 0-22 ZONA 1,77630164,DINA RUBI GONZALEZ DE LEON,FERNANDO DAVID RIVERA ESPAÑA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),QUETZALTENANGO
11,09-01-0139-46,09-006,QUETZALTENANGO,QUETZALTENANGO,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,AV JESUS CASTILLO 1-81 ZONA 3,77611270,PEDRO GERÓNIMO SARAT CHAJ,SIN ESPECIFICAR,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),QUETZALTENANGO


# Exportación dataset limpio

In [30]:
combined_csv.to_csv( "clean_dataset.csv", index=False)